<a href="https://colab.research.google.com/github/Nisum004/Vehicle_tracking_and_counting_YOLO/blob/main/vehicle_detection_and_tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.1 MB/s eta 0:00:00


In [2]:
!pip install ultralytics opencv-python

In [3]:
import torch
from google.colab.patches import cv2_imshow
from collections import defaultdict
import cv2
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
if torch.cuda.is_available():
  device = torch.device('cuda')
elif torch.backends.mps.is_available():
  device = torch.device('mps')
else:
  device = torch.device('cpu')

print(f"Using device: {device}")

Using device: cuda


In [5]:
model = YOLO('yolo11x.pt')

class_list = model.names
print(class_list)

{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microw

In [8]:
from ultralytics import YOLO
import cv2
from collections import defaultdict

# Load YOLO model
model = YOLO('yolo11x.pt')

# Open video
cap = cv2.VideoCapture('/content/traffic4.mp4')

width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv2.CAP_PROP_FPS)

# Output video
output_path = '/content/output_counted_fixed.mp4'
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

# Line coordinates (editable)
x1_line, x2_line = 50, 2000  # small segment of the road
y_line = 2500
line_thickness = 9

# Counting setup
class_counts = defaultdict(int)
track_memory = {}        # {track_id: last_y_position}
counted_ids = set()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model.track(frame, persist=True, classes=[1, 2, 3, 5, 6, 7])

    # Draw line
    cv2.line(frame, (x1_line, y_line), (x2_line, y_line), (0, 0, 255), line_thickness)

    if results[0].boxes.id is not None:
        boxes = results[0].boxes.xyxy.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()
        class_indices = results[0].boxes.cls.int().cpu().tolist()

        for box, track_id, cls_idx in zip(boxes, track_ids, class_indices):
            x1, y1, x2, y2 = map(int, box)
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
            class_name = model.names[cls_idx]

            # Draw visual marks
            cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"{class_name} {track_id}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 0), 4)

            # Track movement
            prev_y = track_memory.get(track_id, None)
            track_memory[track_id] = cy

            # Count when crossing from above to below the line (within segment)
            if prev_y is not None and prev_y < y_line <= cy:
                if x1_line < cx < x2_line and track_id not in counted_ids:
                    counted_ids.add(track_id)
                    class_counts[class_name] += 1

        # Display count summary
        y_offset = 1800
        for cls, count in class_counts.items():
            cv2.putText(frame, f"{cls}: {count}", (100, y_offset),
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 255), 6)
            y_offset += 60

    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()

print(f"✅ Saved stabilized output: {output_path}")



0: 640x384 14 cars, 1 motorcycle, 2 buss, 1 truck, 64.0ms
Speed: 3.3ms preprocess, 64.0ms inference, 13.1ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 14 cars, 1 motorcycle, 3 buss, 45.8ms
Speed: 5.4ms preprocess, 45.8ms inference, 18.7ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 14 cars, 1 motorcycle, 1 bus, 2 trucks, 43.3ms
Speed: 3.6ms preprocess, 43.3ms inference, 23.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 14 cars, 1 motorcycle, 2 buss, 1 truck, 43.3ms
Speed: 3.6ms preprocess, 43.3ms inference, 14.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 14 cars, 1 motorcycle, 2 buss, 1 truck, 43.3ms
Speed: 3.4ms preprocess, 43.3ms inference, 10.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 14 cars, 1 motorcycle, 1 bus, 1 truck, 43.3ms
Speed: 3.3ms preprocess, 43.3ms inference, 10.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 14 cars, 1 motorcycle, 2 buss, 1 truck, 43.3ms
Speed: 3.